In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms

print("--- [START] Phase 1 & 2: Loading Data and Initializing ResNet-18 ---")

# 1. Data Preparation
# Resizing images to 32x32 to ensure compatibility with ResNet architecture
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

print("Downloading and loading Fashion-MNIST dataset...")
trainset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

print(f"Data Loaded: {len(trainset)} training images and {len(testset)} test images.")

# 2. Architecture Definition
print("\nAdapting ResNet-18 architecture (Modified for 1-channel grayscale)...")
# Initializing ResNet-18 without pre-trained weights as per project guidelines
model_resnet = models.resnet18(weights=None)

# Significant Modification: Adapting the first layer for 1-channel grayscale input
model_resnet.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

# Modification: Final fully connected layer adapted for 10 fashion categories
model_resnet.fc = nn.Linear(model_resnet.fc.in_features, 10)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_resnet.parameters(), lr=0.0001)
print("Model initialized. Optimizer: Adam | Loss: CrossEntropy.")

# 3. Training Loop
print("\n--- Phase 3: Training Phase (5 Epochs) ---")
model_resnet.train()
for epoch in range(5):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader):
        optimizer.zero_grad() # Reset gradients for backpropagation
        outputs = model_resnet(inputs) # Forward pass
        loss = criterion(outputs, labels) # Calculate loss
        loss.backward() # Backward pass
        optimizer.step() # Update weights
        running_loss += loss.item()

    print(f"Epoch {epoch + 1}/5 finished. Avg Loss: {running_loss/len(trainloader):.4f}")

# 4. Evaluation Phase
print("\n--- Evaluation: Final Testing ---")
model_resnet.eval() # Set model to evaluation mode
correct = 0
total = 0
with torch.no_grad(): # Disable gradient calculation for efficiency
    for images, labels in testloader:
        outputs = model_resnet(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Output matches the terminology used in Table 2 of the report
print(f"Final Test Accuracy of Model 2: {100 * correct / total:.2f}%")

--- [START] Phase 1 & 2: Loading Data and Initializing ResNet-18 ---


100%|██████████| 26.4M/26.4M [00:01<00:00, 19.7MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 346kB/s]
100%|██████████| 4.42M/4.42M [00:00<00:00, 6.33MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 8.45MB/s]


Data Loaded: 60000 training images and 10000 test images.

Adapting ResNet-18 architecture (Modified for 1-channel grayscale)...
Model initialized. Optimizer: Adam | Loss: CrossEntropy.

--- Phase 3: Training Phase (5 Epochs) ---
Epoch 1/5 finished. Avg Loss: 0.4330
Epoch 2/5 finished. Avg Loss: 0.2889
Epoch 3/5 finished. Avg Loss: 0.2322
Epoch 4/5 finished. Avg Loss: 0.1908
Epoch 5/5 finished. Avg Loss: 0.1598

--- Evaluation: Final Testing ---
Final Test Accuracy of Model 2: 89.18%
